## Python Memory Markdown

Memory managment in Python involves a combination of automatic garbage collection, reference counting, and various internal optimizations to efficiently manage memory allocations and dellocations. Understanding this mechanisms can help developers write more efficient and robust appliations
 - Key concepts in python memory management
 - Memory allocation nd dellocation
 - reference counting
 - the gc module
 - memory managment best practices

In [ ]:
''' Reference counting
this is the primary method Python uses to manage memory. Each object in Python maintains a count of references pointing to it. 
when the reference count drops to zero, the memory occupied by the objectis deallocated
'''

import sys

a=[]
print(sys.getrefcount(a)) ##output : 2 -->(one reference from 'a' and one from getrefcount())

2


In [3]:
import sys

a = []
print("Original refcount:", sys.getrefcount(a))

b = a
print("After b=a, refcount:", sys.getrefcount(a))

c = a
print("After c=a, refcount:", sys.getrefcount(a))


Original refcount: 2
After b=a, refcount: 3
After c=a, refcount: 4


In [4]:
del c
print(sys.getrefcount(a))

3


### - Garbage colection
Python includes a cyclic garbage colector to handle reference cycles. reference cycles occur when objects reference each other, preventing their reference counts from reaching zero

In [5]:
import gc #garbage collector
##enable garbage collection
gc.enable()

In [6]:
gc.disable()

In [7]:
gc.collect()

185

In [8]:
##get garbage collection stats
print(gc.get_stats())

[{'collections': 70, 'collected': 989, 'uncollectable': 0}, {'collections': 6, 'collected': 646, 'uncollectable': 0}, {'collections': 1, 'collected': 185, 'uncollectable': 0}]


In [9]:
##get unrechable objects
print(gc.garbage)

[]


## Memory Maganement 
- Use local vars - 	Easy to free
- Use generators - 	Less memory
- del unused vars - 	Clear early
- with statements - 	Auto resource cleanup
- Monitor with gc, sys, psutil	- Debug memory leaks
- Avoid circular refs - 	Cleaner memory
- Use __slots__ - 	Lighter objects

In [ ]:
##handling circular reference
import gc

class MyObject:
    def __init__(self,name):
        self.name=name
        print(f"Object {self.name} created")

    def __del__(self):
        print(f"Object {self.name} deleted")

##Create circular reference
obj1=MyObject("obj1")
obj2=MyObject("obj2")
obj1.ref=obj2
obj2.ref=obj1

del obj1
del obj2

##Manually trigger the garbage collection
gc.collect()

Object obj1 created
Object obj2 created
Object obj1 deleted
Object obj2 deleted


393

In [4]:
##generators for memeory efficiency
#generators allow you to produce items one at a time, using memory efficiently by only keeping one item at the memory at a time

def generate_num(n):
    for i in range(n):
        yield i

##using the generator
for num in generate_num(100000):
    print(num)
    if num>10:
        break

0
1
2
3
4
5
6
7
8
9
10
11


In [12]:
##Profiling memory usage with tracemalloc

import tracemalloc  ##tracemalloc, a built-in Python module to track memory allocations in your program.
 ##This is super useful when you want to check which part of your code is consuming memory.'''

def create_list():
    return [i for i  in range(10000)] #🔁 This function creates a list of 10,000 integers.🧠 This list takes real memory, not a generator

def main():
    tracemalloc.start() #🚀 Starts tracing memory allocations from this point onward.

    create_list() #📦 Creates that big list (10,000 items), which takes memory.

    snapshot = tracemalloc.take_snapshot() #Take a memory snapshot after your code runs
    top_stats = snapshot.statistics('lineno') #Show which lines used memory

    print("[Top 10]")
    for stat in top_stats[:10]:
        print(stat)

main()

[Top 10]
c:\Anaconda\Lib\tokenize.py:582: size=3522 KiB, count=78233, average=46 B
c:\Anaconda\Lib\ast.py:50: size=3430 KiB, count=47725, average=74 B
<string>:1: size=2199 KiB, count=17592, average=128 B
c:\Anaconda\Lib\site-packages\asttokens\line_numbers.py:64: size=1090 KiB, count=34884, average=32 B
c:\Anaconda\Lib\site-packages\asttokens\line_numbers.py:47: size=787 KiB, count=3773, average=213 B
c:\Anaconda\Lib\site-packages\asttokens\asttokens.py:139: size=466 KiB, count=17049, average=28 B
c:\Anaconda\Lib\site-packages\executing\executing.py:241: size=433 KiB, count=5921, average=75 B
c:\Anaconda\Lib\site-packages\executing\executing.py:209: size=317 KiB, count=3391, average=96 B
c:\Anaconda\Lib\linecache.py:172: size=317 KiB, count=3343, average=97 B
c:\Anaconda\Lib\selectors.py:305: size=288 KiB, count=6, average=48.0 KiB
